# Generate Full Disk images of GOES-R NetCDF files and upload them to GitLab database.

https://gitlab.com/adomakor412/goes-r_fd_image

In [1]:
import sys
from netCDF4 import Dataset
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.basemap import Basemap
from pathlib import Path
from subprocess import Popen
import itertools

from pyproj import Proj
import pyproj
import xarray as xr
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pyresample import image, geometry
import sys
import metpy
import seaborn as sns
sns.set(style="darkgrid")
import pandas as pd
import statsmodels.api as sm
import os
import os.path as op
import glob
from convertdate import gregorian, ordinal
from multiprocessing import Pool

import warnings
#warnings.filterwarnings('ignore')

import logging
logger = logging.getLogger()

/home/radomako/.conda/envs/adomako/lib/python3.9/site-packages/pyresample/bilinear/__init__.py:50: UserWarning: XArray and/or zarr not found, XArrayBilinearResampler won't be available.
  warnings.warn("XArray and/or zarr not found, XArrayBilinearResampler won't be available.")


## Paths

In [2]:
# storage = Path('../goes-r_fd_image/')
# sharkfins_FD_NC = Path('~/scratch/gops/amqpfind/adomako_data/l1b_imagery_sharkfin/')
# caterpillar_FD_NC = Path('~/scratch/gops/amqpfind/adomako_data/l1b_imagery_caterpillar_track/')
storage = '../goes-r_fd_image/'
sharkfins_FD_NC = '~/scratch/gops/amqpfind/adomako_data/l1b_imagery_sharkfin/'
caterpillar_FD_NC = '~/scratch/gops/amqpfind/adomako_data/l1b_imagery_caterpillar_track/'

## Functions for unfiltered cloud mask

In [11]:
def Rad2BT(rad, planck_fk1, planck_fk2, planck_bc1, planck_bc2):
    """Radiances to Brightness Temprature (using black body equation)"""
    
    planck_fk1 = abs(planck_fk1)
    planck_fk2 = abs(planck_fk2)
    planck_bc1 = abs(planck_bc1)
    planck_bc2 = abs(planck_bc2)
    
    invRad = np.array(abs(rad))**(-1)
    arg = (invRad*planck_fk1) + 1.0
    T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2)
    return T

def createUnfilteredPlotArray(ncFile,npFile,npPath):#Filtered Histrogram for cloud clear sky mask
    Tmean= []
    times = []
    for ncf, npf in zip(ncFile, npFile):
        imageBox = np.load(op.join(npPath,npf))
        myFile = xr.open_dataset(op.join(ncPath,ncf))
        planck_fk1 = float(myFile['planck_fk1'].data)
        planck_fk2 = float(myFile['planck_fk2'].data) 
        planck_bc1 = float(myFile['planck_bc1'].data)                       
        planck_bc2 = float(myFile['planck_bc2'].data)     
        T = Rad2BT(imageBox.mean(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)
        tString = ncf[31:38]
        times.append(tString)
        Tmean.append(T)
    return times, Tmean

def listurls(prefix,html):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html.text)
    urllist = [elt['href'] for elt in soup.find_all(href=re.compile(prefix))]
    return urllist

def create_nc_Numpy(ncFile, pathOut):
    myFile = xr.open_dataset(ncFile,engine="netcdf4")
    dat = myFile.metpy.parse_cf('Rad')#myFile['Rad']
    geos = dat.metpy.cartopy_crs

    cartopy_extent_goes = geos.x_limits + geos.y_limits
    pyresample_extent_goes = (cartopy_extent_goes[0],
                                cartopy_extent_goes[2],
                                cartopy_extent_goes[1],
                                cartopy_extent_goes[3])
    goes_params = geos.proj4_params
    rad = dat.data
    
    def normIm(im,gamma=1.0,reverse=False):
        nim = ((im-np.nanmin(im))*(np.nanmax(im)-np.nanmin(im))**(-1))
        if reverse:#want clouds to be white
            nim = (1.0-nim**(gamma))
        return nim
    
    def goes_2_roi(geos_crs, 
               target_extent,
               target_rows,#actual length or base
               target_cols,#actual width or height
               cartopy_target_proj,
               data_key='Rad',
               radius_of_influence=50000):
        """Function that goes from loaded GOES data to data resampled in a projection for an extent"""
        cartopy_source_extent = geos_crs.x_limits + geos_crs.y_limits
        pyresample_source_extent = (cartopy_source_extent[0],
                                    cartopy_source_extent[2],
                                    cartopy_source_extent[1],
                                    cartopy_source_extent[3])
        rad = dat.data
        source_area = geometry.AreaDefinition('GOES-1X', 'Full Disk','GOES-1X', 
                                              geos_crs.proj4_params,
                                              rad.shape[1], rad.shape[0],
                                              pyresample_source_extent)
        area_target_def = geometry.AreaDefinition('areaTest', 'Target Region', 'areaTest',
                                            cartopy_target_proj.proj4_params,
                                            target_rows, target_cols,
                                            target_extent)
        #Read up on recommend class 
        #https://pyresample.readthedocs.io/en/latest/search.html?q=Numpy+Resampler+Bilinear&check_keywords
        
        #can suppress warning for long runs as to not generate an overload on the browser rendering .ipynb
        geos_con_nn = image.ImageContainerNearest(rad, 
                                                source_area, 
                                                radius_of_influence=radius_of_influence)

        # Here we are using pyresample for the remapping
        area_proj_con_nn = geos_con_nn.resample(area_target_def)
        return area_proj_con_nn.image_data
        
    def cartopy_pyresample_toggle_extent(input_extent):
        return np.array(input_extent)[np.array([0,2,1,3])]

    def transform_cartopy_extent(source_extent,source_proj, target_proj):
        target_extent = target_proj.transform_points(source_proj, 
                                                     np.array(source_extent[:2]),
                                                     np.array(source_extent[2:])).ravel()
        # target_extent in 3D, must be in 2D
        return cartopy_pyresample_toggle_extent(np.array(target_extent)[np.array([0,1,3,4])])
    pc = ccrs.PlateCarree()
    mc = ccrs.Mercator()

    # Convert extent from pc to mc (both cylindrical projections)
    extent_pc = [-109.59326, -102.40674, 8.94659, -8.94656]
    
    target_extent_mc_cartopy = transform_cartopy_extent(extent_pc, pc, mc)
    target_extent_mc_pyresample = cartopy_pyresample_toggle_extent(target_extent_mc_cartopy)
    
    roi_rads = goes_2_roi(geos,
               target_extent_mc_pyresample,
               401,1001,
               mc)
    ####
    full_filename = op.join(pathOut,ncFile[:-3])
    np.save(full_filename,roi_rads)
    myFile.close()
    return

def download(url,toPath, saveName):
    cmd = [ 'wget ' + url +' -P ' + toPath +' -O '+ saveName]#if re.search('C07',url)
    #print(cmd)
    pid = Popen(cmd, shell=True)
    pid.communicate()
    return

## Load data

In [4]:
Sat = [16,17]
band = range(7,17)
year = [2021]
month = list(range(1,3))
day = list(range(1,32))
hour = list(range(1,24))

In [5]:
search = list(itertools.product(Sat,\
        band,\
        year,\
        month,\
        day,\
        hour))

## Execution

In [13]:
def execute(chunk, band):
    filelog = open('log_FD_image.txt','w')

    #for SS, bb, yyyy, mm, dd, hr in chunk:
    SS, bb, yyyy, mm, dd, hr = chunk
    bb = band
    SS, bb, yyyy, mm, dd, hr = \
        str(SS).zfill(2),\
        str(bb).zfill(2),\
        str(yyyy).zfill(4),\
        str(mm).zfill(2),\
        str(dd).zfill(2),\
        str(hr).zfill(2)

    calDate = gregorian.date(int(yyyy), int(mm), int(dd))
    DDD = ordinal.from_gregorian(calDate.year, calDate.month, calDate.day)[1]
    DDD = str(DDD).zfill(3)

    #Create Directories    
    cond2 = os.path.exists(f'{storage}/{yyyy}')
    cond3 = os.path.exists(f'{storage}/{yyyy}/{DDD}')
    cond4 = os.path.exists(f'{storage}/{yyyy}/{DDD}/{bb}')

    if not cond2:
        #KEEP CONDITION AS A DYNAMIC F-STRING OTHERWISE VARIABLE IS STATIC, utilize loop
        #make di
            #Create Directoriesrectory per year
        cmd = [f'mkdir {storage}/{yyyy}']
        pid = Popen(cmd, shell=True) 
        pid.communicate()

    if not cond3:
        #KEEP CONDITION AS A DYNAMIC F-STRING OTHERWISE VARIABLE IS STATIC, utilize loop
        #make directory per year
        cmd = [f'mkdir {storage}/{yyyy}/{DDD}']
        pid = Popen(cmd, shell=True) 
        pid.communicate()
        
    if not cond4:
        #KEEP CONDITION AS A DYNAMIC F-STRING OTHERWISE VARIABLE IS STATIC, utilize loop
        #make directory per year
        cmd = [f'mkdir {storage}/{yyyy}/{DDD}/{bb}']
        pid = Popen(cmd, shell=True) 
        pid.communicate()

    try:


        #Not a cd command, use absolute path
        #netcdf = glob.glob(f'~/arcdata/goes/grb/goes{SS}/{yyyy}/{yyyy}_{mm}_{dd}_{DDD}/abi/L1b/RadF/')
        netcdf = glob.glob(f'/arcdata/goes/grb/goes{SS}/{yyyy}/{yyyy}_{mm}_{dd}_{DDD}/abi/L1b/RadF/*')
#         print(yyyy,mm,dd,DDD)
#         print(netcdf)
#         print(len(netcdf))
#         break
        print(netcdf, file=filelog)

        for file in netcdf:
            print(file)
            i = file.find('_s')#+2
            HH = file[i+9: i+11]
            MM = file[i+11: i+13]
            ss = file[i+13: i+15]

            '''
            https://stackoverflow.com/questions/13714454/specifying-and-saving-a-figure-with-exact-size-in-pixels
            '''
            my_dpi = 192
            resolution = 5424

            GOES_R = xr.open_dataset(file)
            GOES_image = GOES_R['Rad']


            planck_fk1 = float(GOES_R['planck_fk1'].data)
            planck_fk2 = float(GOES_R['planck_fk2'].data) 
            planck_bc1 = float(GOES_R['planck_bc1'].data)                       
            planck_bc2 = float(GOES_R['planck_bc2'].data)
            #Need not convert projection: using full image, no pyproj interpolation of region of interest
            Kelvin_GOES_image = Rad2BT(GOES_image, planck_fk1, planck_fk2, planck_bc1, planck_bc2)

            #Call pixelage before rendering
            plt.figure(figsize=(resolution/my_dpi, resolution/my_dpi), dpi=my_dpi)

            fig1 = plt.imshow(Kelvin_GOES_image, interpolation='none', vmin = 180, vmax = 300)
            plt.grid(None)
            plt.axis('off')
            #fig1=plt.imshow(Kelvin_GOES_image,cmap='Greys',interpolation='none',vmin=180,vmax=300)

            #print('ERROR IS NOT HERE')
            
            naming = f'{yyyy}/{DDD}/{bb}/FD_goes{SS}_B{bb}_{yyyy}-{mm}-{dd}_{DDD}_{HH}{MM}{ss}'\
                + 'UTC_5424x5424_plain.png'
            
            if not os.path.isfile(op.join(storage,naming)):
                plt.grid(None)
                fig1.figure.savefig( op.join(storage,naming) )
            
            #print('ERROR IS NOT HERE')
            
            #fig1 = plt.imshow(GOES_image)
            plt.clim(180,300)#Kelvin
            cbar = plt.colorbar(fraction=0.1)
            cbar.ax.set_ylabel('Kelvin')
            plt.grid(None) #Call after the imshow, redraws
            plt.axis('off')#Call after the imshow, redraws; "layer" to previous drawing

            title = f'Brightness Temperature in Kelvin \n Full Disk GOES-{SS} Image Band{bb}'
            ttl = plt.title(f"{title}\n Date {mm}-{dd}, Year {yyyy}, \n Day {DDD}, Time {HH}:{MM}:{ss}",\
                       fontsize="x-large", fontweight='bold', pad = 6.0)
            ttl.set_position([.5, 1.05])

            naming = f'{yyyy}/{DDD}/{bb}/FD_goes{SS}_B{bb}_{yyyy}-{mm}-{dd}_{DDD}_{HH}{MM}{ss}UTC_5424x5424.png'
            if not os.path.isfile(op.join(storage,naming)):
                fig1.figure.savefig(op.join(storage,naming))

            plt.close('all')
            GOES_R.close()

    except ValueError as e:
        print('THERE IS AN ERROR')
        print('\n', file = filelog)
        print(file, file = filelog)
        print(e, file = filelog)
        print('\n', file = filelog)

    filelog.close()
    return

In [ ]:
#agents = 5
#chunksize = 3

agents = 3
#Do bands
# use number of bands as agents
with Pool(processes=agents) as pool:
    #pool.map(execute, search, chunksize=None)
    pool.map(execute, search)